-sandbox
<img width = '500' src="https://databricks.com/wp-content/uploads/2021/06/db-pride-logo.svg" style="float: left: margin: 30px"/>        <img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 10px"/>                

<br>
> # Bronze para Silver

<br>

## 03 - Pedidos de empréstimo

Vamos organizar alguns problemas que está na tabela bronze de Solicitações de Empréstimo.

-sandbox
<h2 style="color:red">Informações</h2>

<br>
- Corrigir a tabela de solicitações de empréstimo ao mesmo tempo em que a amplia
   - por que é útil: a geração de sinalizadores ajudará mais tarde na filtragem e computação de estatísticas
- Utilizando **`hash`** para algumas colunas
- método **`.isin()`** 
- função **`when`**, **`otherwise`**

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Configurações úteis<br>

In [0]:
%run ./Includes/09-ClassroomSetup

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Cleaning up the learning environment... Removed working directory: dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/03_pedidos_emprestimo_psp

WARNING: This notebook was tested on DBR 7.0, but we found DBR 7.3. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

The following variables were defined for you: 
 financeDataPath : dbfs:/mnt/training/finance-org 
 The location of the finance dataset used in this capstone project. 
 
 sourceDataPath : dbfs:/mnt/training/finance-org/solutions 
 The location of the specific datasets used in this capstone project as source data. 
 
 loanRequestsPath : dbfs:/mnt/training/finance-org/solutions/bronze/loan_requests 
 dbfs:/mnt/training/finance-org/solutions/bronze/loan_requests 
 
 investorsPath : dbfs:/mnt/training/finance-org/solutions/bronze/investors 
 dbfs:/mnt/training/finance-org/solutions/bronze/investors 
 
 targetDirectory : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/03_pedidos_emprestimo_psp/silver/loan_requests 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/03_pedidos_emprestimo_psp/silver/loan_requests 
 
 banned_investor_list : [26, 270, 1371] 
 Banned investors 
 The following database were created for you: 
 Now using the database identified by databaseName : 
 thomaz_thomazrossito_com_br_capstone_finance_03_pedidos_emprestimo_psp 
 This is a private, per-notebook, database used to provide isolation from other users and exercises. 
 All done!

Declared the following function:
 realityCheck

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas: <br>

<br>
- Ler a tabela de bronze _Solicitações de Empréstimo_ em **`LoanRequestsPath`**.
- Converter os campos **`request_time`** e **`valid_to`** para timestamp.
- Analise os dados. Existe um campo **`password`** contendo informações confidenciais.
  - Criptografar esses dados antes de realizar a ingestão na camada **`Gold`**.
  - Crie uma nova coluna chamada **`password_hash`** e defina os valores da coluna como <a href = 'https: //spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark. sql.functions.md5 'target =' _ blank '> hash MD5 </a> de senhas.
  - Uma vez que o hash é feito, delete a coluna **`password`**.
- Use Case: Como o módulo de solicitação de empréstimo passou recentemente por uma falha, alguns valores de solicitação são strings vazias em branco.
  - Substitua **`request_amount`** por **`nulos`** e adicione uma nova coluna **`_True/False_`** a través da coluna **`missing_amount`**.
  - Defina o valor **`missing_amount` ** para que indique se o valor foi ou não deixado em branco.
- Use Case: Existem alguns investidores na lista negra. Eles são armazenados em uma lista nativa (não em um DataFrame) chamada **`banned_investor_list`**, que foi pré-carregada.
  - Crie uma nova coluna booleana chamada **`banned_investor`** e indique se o investidor de uma respectiva solicitação está listado em **`banned_investor_list`**.
- Ler a tabela **`Silver`** dos investidores que se encontra em ** `investmentsPath` ** para um DataFrame.
- Junte às Solicitações de Empréstimo em que trabalhou com os investidores usando uma junção à esquerda.
- Use Case: Agora crie uma nova coluna chamada **`loan_alert`** no DataFrame associado. 
  - Defina seu valor como **`red`** se **`credit_score`** for menor ou igual a 1. Caso contrário, defina o valor como **`green`**.
- Grave o DataFrame resultante como uma tabela Delta em **`targetDirectory`**.
- Retorne o DataFrame final.

<p> O esquema do DataFrame deve ser:
  
|name|type|
|---|---|
|investor_id|LongType|
|loan_id|LongType|
|product_id|LongType|
|request_amount|StringType|
|request_length|LongType|
|request_time|TimestampType|
|valid_to|TimestampType|
|password_hash|StringType|
|missing_amount|BooleanType|
|banned_investor|BooleanType|
|loan_alert|StringType|

In [0]:
def func_pedidoEmprestimo(spark, loanRequestsPath, investorsPath, targetDirectory):

  
  from pyspark.sql.functions import col, md5, when, trim
  
  # Ler a tabela bronze de Solicitações de Empréstimo em a partir de `LoanRequestsPath`
  # Converta os campos "request_time" e "valid_to" em timestamp
  # Criando uma nova coluna chamada `password_hash` e defina os valores da coluna para hash MD5. 
  # Deletando a coluna `password`.
  # Valores de solicitação são strings vazias em branco. Substitua-os por nulos e adicione uma nova coluna True/False chamada `missing_amount`
  # Crie uma nova coluna booleana chamada `banned_investor` e indique se o investidor da respectiva solicitação é um investidor na lista negra.
  pedidoEmprestimo = (spark.read
                           .format("delta")
                           .load(loanRequestsPath)
                           .withColumn("request_time", col("request_time").cast("timestamp"))
                           .withColumn("valid_to", col("valid_to").cast("timestamp"))
                           .withColumn("password_hash", md5(col("password")))
                           .drop(col("password"))
                           .withColumn("request_amount", when(col("request_amount") == "", None).otherwise(col("request_amount")))
                           .withColumn("missing_amount", col("request_amount").isNull())
                           .withColumn("banned_investor", col("investor_id").isin(banned_investor_list)))
  
  
  # Ler a tabela Silver de investidores a partir de "investorsPath" para um DataFrame.
  investidores = (spark.read
                       .format("delta")
                       .load(investorsPath)
                       .select("investor_id",
                               "credit_score"))

  
  # Junte às Solicitações de Empréstimo com os Investidores usando uma junção à esquerda.
  # Crie uma nova coluna chamada `loan_alert` no DataFrame associado.
    # Defina seu valor como "vermelho" se o `credit_score` for menor ou igual a 1. Caso contrário, defina o valor como" verde "
  pedidoEmprestimoDF = (pedidoEmprestimo.join(investidores, on="investor_id", how="left")
                                        .withColumn("loan_alert", when(col("credit_score") <= 1, "red").otherwise("green"))
                                        .drop("credit_score"))
  
  
  # Grave o DataFrame resultante como uma tabela Delta em `targetDirectory` com modo de substituição e opção de esquema de substituição
  pedidoEmprestimoDF.write \
                    .format("delta") \
                    .mode("overwrite") \
                    .option("overwriteSchema", True) \
                    .save(targetDirectory)
  
  
  # Retorna o DataFrame Final
  return pedidoEmprestimoDF

In [0]:
finalDF = func_pedidoEmprestimo(spark, loanRequestsPath, investorsPath, targetDirectory)

In [0]:
display(finalDF)

investor_id,loan_id,product_id,request_amount,request_length,request_time,valid_to,password_hash,missing_amount,banned_investor,loan_alert
747,503830834,1,$ 11918,45,2019-09-05T00:24:40.000+0000,2019-09-20T00:24:40.000+0000,73ce6846caf83bab704f51f291b5dfd2,false,false,green
817,503830835,1,$ 10408,222,2019-09-18T00:01:03.000+0000,2019-10-03T00:01:03.000+0000,e4d3baca1bb213f7949a662b52c09f96,false,false,green
1124,503830836,1,$ 12409,235,2019-09-23T08:05:24.000+0000,2019-10-08T08:05:24.000+0000,02735c41e55a1bc5ed1f3d563b1aed2e,false,false,green
1157,503830837,1,$ 11332,58,2019-09-26T10:32:10.000+0000,2019-10-11T10:32:10.000+0000,d63a9116eb1144e4a8a0ff90b76527c4,false,false,red
1148,503830838,1,$ 5803,41,2019-09-18T13:14:27.000+0000,2019-10-03T13:14:27.000+0000,cc157aa9ee236a1578124ae32a875b53,false,false,green
983,503830839,1,$ 14673,52,2019-09-04T14:04:04.000+0000,2019-09-19T14:04:04.000+0000,8fa7909f3d2ba3fa177362d58870d056,false,false,green
194,503830840,1,$ 7358,224,2019-09-16T23:15:01.000+0000,2019-10-01T23:15:01.000+0000,6365cac9a4367339c9321eb3c8d40b67,false,false,green
802,503830841,1,$ 13598,100,2019-09-09T10:31:40.000+0000,2019-09-24T10:31:40.000+0000,f36313b79acda175b223ae4520d95f7e,false,false,green
1160,503830842,1,$ 6657,34,2019-09-03T13:34:39.000+0000,2019-09-18T13:34:39.000+0000,76c345524d2573641a5d3bff070dbd7b,false,false,green
323,503830843,1,$14959,184,2019-09-23T01:44:33.000+0000,2019-10-08T01:44:33.000+0000,e610126e4133c93ab84edd5d7bfcc5f1,false,false,red


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_pedidoEmprestimo, spark, loanRequestsPath, investorsPath, targetDirectory)

Points,Test,Result
1,Returns correct schema,
1,Returns DataFrame with correct number of rows,
1,Returns DataFrame with correct results,
1,Silver table in place,
1,Silver table has correct content,


## <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Próximo <p>

> * [Pedidos de empréstimo 2]($./04-Pedidos emprestimo 2)

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>